In [1]:
import pandas as pd
import numpy as np
from subprocess import Popen, PIPE, check_output, run
import os
import sys
from io import BytesIO
from sklearn.model_selection import train_test_split

In [40]:
#load the admission ids and subject ids
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/'
positive = pd.read_csv(path + '1.0/ids/hadm_ids_positive.txt')
all_cases = pd.read_csv(path + '1.0/ids/hadm_ids_all.txt')
negative = pd.read_csv(path + '1.0/ids/hadm_ids_negative.txt')
admissions = pd.read_csv(path+'1.0/core/admissions.csv')

In [41]:
positive_subjects = admissions['subject_id'][admissions['hadm_id'].isin(positive['hadm_id'])]
negative_subjects = admissions['subject_id'][admissions['hadm_id'].isin(negative['hadm_id'])]

In [42]:
cxr_use = pd.read_csv(path+'1.0/cxr_metadata_use2.csv')
#create the naming logic needed to download
cxr_use['folder'] = 'p' + cxr_use['subject_id'].astype(str).str[:2]
cxr_use['sub_folder'] = 'p' + cxr_use['subject_id'].astype(str)
cxr_use['study_folder'] = 's' + cxr_use['study_id'].astype(str)
cxr_use.head()

,dicom_id,subject_id,study_id,PerformedProcedureStepDescription,ViewPosition,Rows,Columns,StudyDate,StudyTime,ProcedureCodeSequence_CodeMeaning,...,first_careunit,last_careunit,intime,outtime,los,obs_window_end,obs_window_start,folder,sub_folder,study_folder
0,469d0d94-3dad5068-efac76ef-a28cc502-68fe6275,10001884,50376803,CHEST (PORTABLE AP),AP,2539,2517,2131-01-15,44509.078,CHEST (PORTABLE AP),...,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2131-01-11 04:20:05,2131-01-20 08:27:30,9.171817,2131-01-18 04:20:05,2131-01-08 04:20:05,p10,p10001884,s50376803
1,1e647043-eed3576e-3123c170-780cb897-93a89502,10002013,52163036,Performed Desc,PA,1865,1789,2160-05-21,133230.000,CHEST (PA AND LAT),...,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2160-05-18 10:00:53,2160-05-19 17:33:33,1.314352,2160-05-25 10:00:53,2160-05-15 10:00:53,p10,p10002013,s52163036
2,7f51d06c-dad16fe6-ff21a632-edc5a3e6-ffa387bf,10002428,50292543,CHEST (PORTABLE AP),AP,3056,2544,2156-05-18,24737.203,CHEST (PORTABLE AP),...,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,2156-05-18 14:49:34,2156-05-08 14:49:34,p10,p10002428,s50292543
3,c25b9023-9cd26369-dbe2b67b-c250ef52-b772137e,10003019,55487822,CHEST (PORTABLE AP),AP,2544,3056,2175-10-11,153444.859,CHEST (PORTABLE AP),...,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,2175-10-15 18:58:00,2175-10-05 18:58:00,p10,p10003019,s55487822
4,4057348e-68278864-98cc1e88-41ae8009-264cb3bf,10003400,58820563,CHEST (PORTABLE AP),AP,3050,2539,2137-08-24,32422.562,CHEST (PORTABLE AP),...,Coronary Care Unit (CCU),Medical Intensive Care Unit (MICU),2137-08-17 17:36:37,2137-09-02 19:17:11,16.069838,2137-08-24 17:36:37,2137-08-14 17:36:37,p10,p10003400,s58820563


In [43]:
cxr_use_positive = cxr_use[cxr_use['subject_id'].isin(positive_subjects)]
cxr_use_negative = cxr_use[cxr_use['subject_id'].isin(negative_subjects)]

In [44]:
#create file request
request = 'mv '
files = ''
for i, row in cxr_use_positive.iterrows():
    file = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/physionet.org/files/mimic-cxr-jpg/2.0.0/files/{}/{}/{}/{}.jpg '.format(
        row['folder'], row['sub_folder'], row['study_folder'], row['dicom_id'])
    files = files + file
new_loc = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/train/POSITIVE/ '
files = files + new_loc

positive_request = request+files

In [46]:
#create file request
request = 'mv '
files = ''
for i, row in cxr_use_negative.iterrows():
    file = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/physionet.org/files/mimic-cxr-jpg/2.0.0/files/{}/{}/{}/{}.jpg '.format(
        row['folder'], row['sub_folder'], row['study_folder'], row['dicom_id'])
    files = files + file
new_loc = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/train/NEGATIVE/ '  
files = files  + new_loc
 
negative_request = request+files

## filter based on those with images

In [48]:
#identify which ones got images downloaded
request = 'ls /gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/train/POSITIVE'
request = request.split(' ')
positive_downloaded = run(request, stdout=PIPE)

In [49]:
def decode(row):
    return row['dicom_id'].decode("utf-8").replace('.jpg\n','')

In [50]:
positive_dl = pd.DataFrame(BytesIO(positive_downloaded.stdout), columns = ['dicom_id'])
positive_dl['dicom_id'] = positive_dl.apply(decode, axis = 1)

In [51]:
#identify which ones got images downloaded
request = 'ls /gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/train/NEGATIVE'
request = request.split(' ')
negative_downloaded = run(request, stdout=PIPE)

In [52]:
negative_dl = pd.DataFrame(BytesIO(negative_downloaded.stdout), columns = ['dicom_id'])
negative_dl['dicom_id'] = negative_dl.apply(decode, axis = 1)

In [53]:
## find the subjects 
positive_cases = cxr_use_positive.merge(positive_dl, on = 'dicom_id')
negative_cases = cxr_use_negative.merge(negative_dl, on = 'dicom_id')

positive_cases['outcome'] = [1 for i in range(positive_cases.shape[0])]
negative_cases['outcome'] = [0 for i in range(negative_cases.shape[0])]

In [55]:
## split the positive and negative cases
positive_train, positive_test = train_test_split(positive_cases, test_size=0.2, random_state=42)
negative_train, negative_test = train_test_split(negative_cases, test_size=0.2, random_state=42)

In [56]:
positive_train['test'] = [0 for i in range(positive_train.shape[0])]
positive_test['test'] = [1 for i in range(positive_test.shape[0])]
negative_train['test'] = [0 for i in range(negative_train.shape[0])]
negative_test['test'] = [1 for i in range(negative_test.shape[0])]

In [57]:
df = pd.concat([positive_train, positive_test,
                negative_train, negative_test])[['dicom_id', 'subject_id',
                                                 'outcome', 'test']]


In [58]:
#Get the dicomids and download the specific images
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/'
icustays = pd.read_csv(path + 'small/icu/icustays.csv', parse_dates = ['intime', 'outtime'])
icustays = icustays.loc[icustays.groupby('hadm_id').los.idxmax()]

In [59]:
df = df.merge(icustays[['subject_id', 'hadm_id']], on = 'subject_id')

In [60]:
df = df[['hadm_id','subject_id','dicom_id', 'outcome', 'test']]

In [62]:
df.to_csv('/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/train_test_split.csv', index = False)

## move train and test set

In [2]:
path = '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/'
split = pd.read_csv(path + '1.0/small/train_test_split.csv')

In [4]:
split.head()

,hadm_id,subject_id,dicom_id,outcome,test
0,24483629,15811456,ce8c0e78-1ce678ad-6d8cb574-827f203b-2dfa61a4,1,0
1,26021426,15811456,ce8c0e78-1ce678ad-6d8cb574-827f203b-2dfa61a4,1,0
2,29271096,15811456,ce8c0e78-1ce678ad-6d8cb574-827f203b-2dfa61a4,1,0
3,20054615,13903940,9d6a4faa-b6ce6b32-16d5beb3-d66b79ec-f9385d12,1,0
4,29093665,15588817,a35bbcd9-523ca0c1-cc05a064-7e24059c-15de0862,1,0


In [5]:
train_pos = split[(split['outcome']==1) & (split['test']==0)]
train_neg = split[(split['outcome']==0) & (split['test']==0)]
test_pos = split[(split['outcome']==1) & (split['test']==1)]
test_neg = split[(split['outcome']==0) & (split['test']==1)]

In [12]:
##positive
command = 'mv '
for value in test_pos['dicom_id']:
    value = value + '.jpg '
    command = command + value
command = command + '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/test/POSITIVE' 

In [15]:
##negative
command = 'mv '
for value in test_neg['dicom_id']:
    value = value + '.jpg '
    command = command + value
command = command + '/gpfs/commons/groups/gursoy_lab/aelhussein/ML_project/1.0/small/cxr/test/NEGATIVE' 